In [0]:
fastq_files = !find /gpfs_fs/home/eckertlab/projects/burt/seq/dedupe -name "*.fastq"

In [0]:
len(fastq_files)

In [0]:
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from ipyparallel import Client

In [0]:
rc = Client(profile="sge")

In [0]:
dv =rc[:]
lv = rc.load_balanced_view()
len(dv)

In [0]:
with dv.sync_imports():
    from Bio.SeqIO.QualityIO import FastqGeneralIterator

In [0]:
def count_records(f):
    recs = 0
    for name, seq, qual in FastqGeneralIterator(open(f)):
        recs += 1
    return f, recs

In [0]:
dv['count_records'] = count_records

In [0]:
jobs = lv.map_async(count_records, fastq_files)

In [0]:
jobs.progress

In [0]:
for j in jobs:
    print(j)

In [0]:
import pandas as pd
import numpy as np
import os

In [0]:
counts = pd.DataFrame([x for x in jobs], columns = ['file', 'reads'])

In [0]:
counts["sample_name"] = counts.file.apply(lambda x: os.path.basename(x).replace(".R1.fastq", ""))
counts['population'] = counts.sample_name.apply(lambda x: "-".join(x.split("-")[0:-1]))
counts['species'] = counts.sample_name.apply(lambda x: (x.split("-")[0]))
counts[['reads', 'species']].groupby("species").describe()